In [107]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

# Variáveis de ambiente
load_dotenv()
host = os.getenv('host')
user = os.getenv('user')
password = os.getenv('password')
database = os.getenv('database')
port = os.getenv('port')

db_user = user     
db_password = password
db_host = host  
db_port = port       
db_name = database     

# String de conexão para MySQL usando PyMySQL
db_string = f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Criando a engine do SQLAlchemy
engine = create_engine(db_string)

In [108]:
pd.read_sql_query('SELECT * FROM online_retail_table LIMIT 3', con=engine)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom


### Dicionário de Variáveis
- InvoiceNo: Número da Fatura. Nominal. Um número inteiro de 6 dígitos atribuído de forma única a cada transação. Se este código começar com a letra 'c', indica um cancelamento.
- StockCode: Código do Produto. Nominal. Um número inteiro de 5 dígitos atribuído de forma única a cada produto distinto.
- Description: Descrição do Produto. Nominal. Nome do produto (item).
- Quantity: Quantidade. Numérico. Quantidade de cada produto por transação.
- InvoiceDate: Data e Hora da Fatura. Numérico. O dia e a hora em que a transação foi gerada.
- UnitPrice: Preço Unitário. Numérico. Preço do produto por unidade em libras esterlinas (£).
- CustomerID: Número do Cliente. Nominal. Um número inteiro de 5 dígitos atribuído de forma única a cada cliente.
- Country: País. Nominal. Nome do país onde o cliente reside.

### Perguntas de Négocios
1 -  Quais produtos são os mais vendidos? Como podemos identificar os itens mais populares entre os clientes para garantir que o estoque desses produtos esteja sempre disponível?

In [109]:
query = '''
SELECT STOCKCODE, DESCRIPTION, SUM(QUANTITY) AS TOTAL_VENDIDO
FROM online_retail_table
WHERE INVOICENO NOT LIKE 'C%%'
AND STOCKCODE REGEXP '^[0-9]+$'
GROUP BY STOCKCODE, DESCRIPTION
ORDER BY TOTAL_VENDIDO DESC
LIMIT 10
'''
pd.read_sql_query(query, con=engine)

,STOCKCODE,DESCRIPTION,TOTAL_VENDIDO
0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,109169.0
1,23843,"PAPER CRAFT , LITTLE BIRDIE",80995.0
2,84879,ASSORTED COLOUR BIRD ORNAMENT,79913.0
3,23166,MEDIUM CERAMIC TOP STORAGE JAR,77916.0
4,17003,BROCADE RING PURSE,71129.0
5,21977,PACK OF 60 PINK PAISLEY CAKE CASES,55270.0
6,84991,60 TEATIME FAIRY CAKE CASES,53495.0
7,21212,PACK OF 72 RETROSPOT CAKE CASES,46107.0
8,21212,PACK OF 72 RETRO SPOT CAKE CASES,45156.0
9,22197,SMALL POPCORN HOLDER,43970.0


- Os produtos mais vendidos, conforme a análise de vendas, incluem itens como WORLD WAR 2 GLIDERS ASSTD DESIGNS, com 109.169 unidades vendidas, seguido por PAPER CRAFT, LITTLE BIRDIE, com 80.995 unidades, e ASSORTED COLOUR BIRD ORNAMENT, com 79.913 unidades. Outros produtos populares incluem MEDIUM CERAMIC TOP STORAGE JAR e BROCADE RING PURSE, com 77.916 e 71.129 unidades vendidas, respectivamente. Além disso, há destaque para várias variações de caixas de bolo, como a PACK OF 60 PINK PAISLEY CAKE CASES (55.270 unidades), 60 TEATIME FAIRY CAKE CASES (53.495 unidades) e PACK OF 72 RETROSPOT CAKE CASES (46.107 e 45.156 unidades vendidas), além do SMALL POPCORN HOLDER, que atingiu 43.970 unidades.
- Para garantir que o estoque desses itens esteja sempre disponível e atender à demanda dos clientes, é fundamental identificar regularmente esses produtos mais vendidos. Ao fazer isso, a empresa pode evitar rupturas de estoque, melhorar a experiência do cliente, aumentar as vendas e minimizar os custos associados a excessos ou faltas de produtos em estoque.

2 -  Quais os produtos mais comprados em diferentes períodos do ano? Como a sazonalidade afeta a demanda por determinados produtos e como podemos otimizar nosso inventário com base nessas informações?

In [110]:
query = '''
SELECT MONTH, STOCKCODE, DESCRIPTION, TOTAL_VENDA_MES 
FROM (
SELECT EXTRACT(MONTH FROM INVOICEDATE) AS MONTH, STOCKCODE, DESCRIPTION, 
SUM(QUANTITY) AS TOTAL_VENDA_MES,
RANK() OVER (PARTITION BY EXTRACT(MONTH FROM INVOICEDATE)ORDER BY SUM(QUANTITY) DESC) AS RANKING
FROM online_retail_table
WHERE INVOICENO NOT LIKE 'C%%'
AND STOCKCODE REGEXP '^[0-9]+$'
GROUP BY MONTH, STOCKCODE, DESCRIPTION
) AS TOP_PRODUTOS_MES
WHERE RANKING = 1
ORDER BY MONTH
'''
pd.read_sql_query(query, con=engine)

,MONTH,STOCKCODE,DESCRIPTION,TOTAL_VENDA_MES
0,1,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215.0
1,2,37410,BLACK AND WHITE PAISLEY FLOWER MUG,19248.0
2,3,21091,SET/6 WOODLAND PAPER PLATES,13099.0
3,4,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,13968.0
4,5,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,8080.0
5,6,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,7308.0
6,7,15036,ASSORTED COLOURS SILK FAN,6813.0
7,8,84879,ASSORTED COLOUR BIRD ORNAMENT,9564.0
8,9,17003,BROCADE RING PURSE,16006.0
9,10,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,13790.0


- A análise dos produtos mais vendidos ao longo dos meses do ano revela que certos itens têm maior demanda em diferentes períodos, destacando-se padrões sazonais claros. Por exemplo, o MEDIUM CERAMIC TOP STORAGE JAR foi o mais vendido em janeiro, com 74.215 unidades vendidas. No mês de fevereiro, o destaque foi para o BLACK AND WHITE PAISLEY FLOWER MUG, com 19.248 unidades, e em março, o item mais vendido foi o SET/6 WOODLAND PAPER PLATES, com 13.099 unidades. O WORLD WAR 2 GLIDERS ASSTD DESIGNS apresentou picos de vendas nos meses de abril, maio, junho e outubro, variando de 8.080 a 13.968 unidades vendidas, mostrando consistência de vendas ao longo desses meses. Outro item com bom desempenho foi o ASSORTED COLOUR BIRD ORNAMENT, que se destacou nos meses de agosto e novembro, com vendas de 9.564 e 14.389 unidades, respectivamente. Além disso, o BROCADE RING PURSE obteve 16.006 unidades vendidas em setembro. Em dezembro, o PAPER CRAFT, LITTLE BIRDIE liderou as vendas com 80.995 unidades, o que pode indicar uma tendência sazonal associada a eventos ou feriados.
- Com base nessas informações, é possível otimizar o inventário, ajustando os níveis de estoque de acordo com a sazonalidade. Produtos que apresentam picos em determinados meses devem ter um planejamento de estoque mais robusto nesses períodos, garantindo que a demanda seja atendida sem gerar falta de produtos ou excesso de estoque.

3 - Quais são os períodos de pico de vendas? Quais dias têm maior volume de vendas? Podemos ajustar nossa estratégia de marketing ou oferta de produtos durante esses períodos?

In [111]:
query = '''
SELECT EXTRACT(DAY FROM INVOICEDATE) AS DAY,
SUM(QUANTITY) AS TOTAL_VENDA_DIA
FROM online_retail_table
WHERE INVOICENO NOT LIKE 'C%%'
AND STOCKCODE REGEXP '^[0-9]+$'
GROUP BY DAY
ORDER BY TOTAL_VENDA_DIA DESC
LIMIT 10
'''
pd.read_sql_query(query, con=engine)

,DAY,TOTAL_VENDA_DIA
0,9,454606.0
1,7,406097.0
2,15,393651.0
3,18,390308.0
4,17,387291.0
5,21,352784.0
6,8,343226.0
7,5,340606.0
8,14,330871.0
9,11,326611.0


- A análise dos dias com maior volume de vendas revela que os picos ocorrem nos dias 9, 7, 15, e 18, com valores de vendas totais de 454.606, 406.097, 393.651, e 390.308, respectivamente. Esses dias claramente apresentam uma demanda significativamente maior, sugerindo que há padrões consistentes de aumento de vendas em torno dessas datas específicas. Outros dias com vendas expressivas incluem o dia 17, com 387.291 em vendas, e o dia 21, com 352.784. Esses padrões podem indicar influências sazonais ou eventos específicos que impulsionam a demanda em determinados dias do mês.
- Para maximizar o potencial de vendas nesses dias de pico, seria interessante ajustar as estratégias de marketing e oferta de produtos. Isso pode incluir campanhas promocionais focadas nesses períodos, o aumento do inventário dos produtos mais vendidos e a intensificação de esforços de marketing, como campanhas publicitárias ou descontos especiais. Aproveitar esses picos de demanda garante que a empresa esteja preparada para atender ao aumento de vendas sem correr o risco de esgotar o estoque ou perder oportunidades de faturamento.
- Além disso, a antecipação e planejamento logístico adequados para esses dias mais movimentados pode evitar atrasos ou problemas operacionais, assegurando uma experiência positiva para os clientes e o aproveitamento máximo do potencial de vendas.

4 - Quais clientes são responsáveis pelas maiores receitas? Quem são os clientes que mais gastam com base no valor total de suas compras?

In [112]:
query = '''
SELECT CUSTOMERID,
ROUND(SUM(QUANTITY * UNITPRICE),2) AS RECEITA_TOTAL
FROM online_retail_table
WHERE INVOICENO NOT LIKE 'C%%'
AND STOCKCODE REGEXP '^[0-9]+$'
GROUP BY CUSTOMERID
ORDER BY RECEITA_TOTAL DESC
LIMIT 10
'''
pd.read_sql_query(query, con=engine)

,CUSTOMERID,RECEITA_TOTAL
0,18102,553408.56
1,14646,480743.46
2,14156,278736.67
3,14911,248941.83
4,17450,231106.41
5,16446,168472.49
6,17511,152913.47
7,12415,139251.43
8,13694,134351.39
9,15061,131118.19


- A análise dos clientes responsáveis pelas maiores receitas revela que o CustomerID 18102 é o cliente que mais gasta, seguido pelos CustomerID 14646, CustomerID 14156, CustomerID 14911, e CustomerID 17450. Esses clientes destacam-se como os principais responsáveis por uma grande parte da receita total.
- Esses clientes representam uma parcela significativa da receita da empresa, indicando que são clientes de alto valor. Eles podem ser considerados "clientes VIP", e a empresa pode adotar estratégias específicas para manter esses consumidores satisfeitos e incentivá-los a continuar comprando. Programas de fidelidade, descontos exclusivos e atendimento personalizado são algumas das táticas que podem ser implementadas para reforçar a relação com esses clientes de alto valor.
- Além disso, acompanhar de perto o comportamento de compra desses clientes pode ajudar a desenvolver ações de marketing mais eficazes e direcionadas, maximizando o retorno e garantindo que esses clientes continuem a gerar receitas significativas

5 - Existem produtos com baixa rotatividade e valor unitário reduzido que deveriam ser descontinuados? Produtos com baixo volume de vendas e preço inferior podem comprometer a eficiência do inventário e, por isso, devem ser reavaliados. Para este cenário, consideraremos produtos que venderam menos de 30 unidades nos últimos dois anos e cujo valor unitário seja inferior a 1000 dólares.

In [113]:
query = '''
SELECT STOCKCODE, DESCRIPTION, UNITPRICE, SUM(QUANTITY) AS TOTAL_VENDIDO
FROM online_retail_table
WHERE INVOICENO NOT LIKE 'C%%'
AND STOCKCODE REGEXP '^[0-9]+$'
AND UNITPRICE <1000
GROUP BY STOCKCODE, DESCRIPTION, UNITPRICE 
HAVING SUM(QUANTITY) < 30
ORDER BY TOTAL_VENDIDO
'''
df_produtos_baixa_rotacao = pd.read_sql_query(query, con=engine)

In [114]:
df_produtos_baixa_rotacao

,STOCKCODE,DESCRIPTION,UNITPRICE,TOTAL_VENDIDO
0,35995,SET/6 BLACK GLITTER REINDEER CANDLE,2.10,1.0
1,84648,NEW BAROQUE WALL MIRROR,12.75,1.0
2,21767,FRENCH STYLE WIRE DOOR CABINET,29.95,1.0
3,21838,DADDY MOUSE RED GINGHAM BOW TIE,5.45,1.0
4,47556,SET/2 SPOTTY + ROSE TEA TOWELS,2.55,1.0
...,...,...,...,...
2165,21156,RETROSPOT CHILDRENS APRON,4.13,29.0
2166,23543,KEEP CALM WALL ART,8.25,29.0
2167,23466,TUSCAN VILLA DOVECOTE BIRD FEEDER,8.25,29.0
2168,22313,OFFICE MUG WARMER PINK,1.25,29.0


- A análise de produtos com baixa rotatividade e valor unitário reduzido revela que cerca de 2.000 produtos caem nessa categoria, tendo vendido menos de 30 unidades nos últimos dois anos e com preço unitário inferior a 1.000 dólares. Exemplos desses produtos incluem o SET/6 BLACK GLITTER REINDEER CANDLE com apenas 1 unidade vendida a 2,10 dólares, o NEW BAROQUE WALL MIRROR também com 1 unidade vendida a 12,75 dólares, e o DADDY MOUSE RED GINGHAM BOW TIE com apenas 1 unidade vendida a 5,45 dólares.
- Esses itens, com baixa rotatividade e baixo valor unitário, podem estar ocupando espaço desnecessário no estoque e comprometendo a eficiência da operação. Manter produtos com vendas tão reduzidas pode resultar em custos adicionais com armazenagem e gerenciamento de inventário. Portanto, é recomendável reavaliar esses itens para considerar sua descontinuação ou substituição por produtos com maior demanda e margem de lucro mais significativa.
- Ao descontinuar ou revisar esses produtos de baixo desempenho, a empresa pode otimizar seu inventário, liberando espaço e recursos para focar em itens de maior demanda e valor.